In [1]:
import pyspark

In [37]:
import pandas as pd
import numpy as np
from pydataset import data
from pyspark.sql.functions import col, expr, lit, regexp_extract, regexp_replace, avg, month, sum, max, year

In [3]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/06 21:05:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [4]:
languages = {"language":["Python", "Java", "Ruby", "JavaScript"]}

In [5]:
df = spark.createDataFrame(pd.DataFrame(languages))

In [6]:
def pyspark_shape(df):
    print(f"{df.count()}, {len(df.columns)}")

In [7]:
pyspark_shape(df)

4, 1


In [8]:
df.show(7)

+----------+
|  language|
+----------+
|    Python|
|      Java|
|      Ruby|
|JavaScript|
+----------+



# 2.Load the mpg dataset as a spark dataframe.

- Create 1 column of output that contains a message like the one below:


`The 1999 audi a4 has a 4 cylinder engine.`
For each vehicle.

In [9]:
mpg = spark.createDataFrame(data("mpg"))

In [10]:
mpg.first()['year'], mpg.first()['manufacturer'], mpg.first()['model'], mpg.first()['cyl']

(1999, 'audi', 'a4', 4)

- Transform the trans column so that it only contains either manual or auto.

In [11]:
mpg.filter((mpg.trans.startswith('m'))|(mpg.trans.startswith('a'))).show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



# 3. Load the tips dataset as a spark dataframe.

In [12]:
tips = spark.createDataFrame(data("tips"))

In [13]:
pyspark_shape(tips)

244, 7


- What percentage of observations are smokers?

In [14]:
round(tips.filter(tips.smoker.startswith("Y")).count()/tips.count(),3)

0.381

- Create a column that contains the tip percentage

In [15]:
tips = tips.withColumn("tip_percentage", tips.tip/tips.total_bill)

In [16]:
tips.show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



- Calculate the average tip percentage for each combination of sex and smoker.

In [17]:
tips.groupBy("smoker").agg(avg("tip_percentage")).collect()

[Row(smoker='No', avg(tip_percentage)=0.15932846217921526),
 Row(smoker='Yes', avg(tip_percentage)=0.16319604463687792)]

In [18]:
tips.groupBy("sex").agg(avg("tip_percentage")).collect()

[Row(sex='Female', avg(tip_percentage)=0.16649073632892478),
 Row(sex='Male', avg(tip_percentage)=0.15765054700429743)]

# 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [19]:
from vega_datasets import data

In [20]:
weather = data.seattle_weather().assign(data=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+-------------------+-------------+--------+--------+----+-------+----------+
|               date|precipitation|temp_max|temp_min|wind|weather|      data|
+-------------------+-------------+--------+--------+----+-------+----------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|2012-01-01|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|2012-01-02|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|2012-01-03|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|2012-01-04|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|2012-01-05|
|2012-01-06 00:00:00|          2.5|     4.4|     2.2| 2.2|   rain|2012-01-06|
+-------------------+-------------+--------+--------+----+-------+----------+
only showing top 6 rows



- Convert the temperatures to fahrenheit.

In [21]:
weather = weather.withColumn("f_temp_max", (weather.temp_max * 1.8) + 32)

In [22]:
weather = weather.withColumn("f_temp_min", (weather.temp_min * 1.8) + 32)

In [23]:
weather.show(5)

+-------------------+-------------+--------+--------+----+-------+----------+------------------+----------+
|               date|precipitation|temp_max|temp_min|wind|weather|      data|        f_temp_max|f_temp_min|
+-------------------+-------------+--------+--------+----+-------+----------+------------------+----------+
|2012-01-01 00:00:00|          0.0|    12.8|     5.0| 4.7|drizzle|2012-01-01|55.040000000000006|      41.0|
|2012-01-02 00:00:00|         10.9|    10.6|     2.8| 4.5|   rain|2012-01-02|             51.08|     37.04|
|2012-01-03 00:00:00|          0.8|    11.7|     7.2| 2.3|   rain|2012-01-03|             53.06|     44.96|
|2012-01-04 00:00:00|         20.3|    12.2|     5.6| 4.7|   rain|2012-01-04|             53.96|     42.08|
|2012-01-05 00:00:00|          1.3|     8.9|     2.8| 6.1|   rain|2012-01-05|48.019999999999996|     37.04|
+-------------------+-------------+--------+--------+----+-------+----------+------------------+----------+
only showing top 5 rows



- Which month has the most rain, on average?

In [47]:
yr_wind = weather.groupBy(month("date").alias("month")).agg(sum("precipitation").alias("precipitation")).show()

+-----+------------------+
|month|     precipitation|
+-----+------------------+
|   12|             622.7|
|    1|465.99999999999994|
|    6|             132.9|
|    3|             606.2|
|    5|             207.5|
|    9|235.49999999999997|
|    4|             375.4|
|    8|             163.7|
|    7|              48.2|
|   10|             503.4|
|   11|             642.5|
|    2|             422.0|
+-----+------------------+



- Which year was the windiest?

In [45]:
yr_wind = weather.groupBy(year("date").alias("year")).agg(sum("wind").alias("wind")).show()

+----+------------------+
|year|              wind|
+----+------------------+
|2012|1244.6999999999998|
|2013|1100.8000000000002|
|2014|1236.5000000000005|
|2015|            1153.3|
+----+------------------+



- What is the most frequent type of weather in January?

- What is the average high and low temperature on sunny days in July in 2013 and 2014?

- What percentage of days were rainy in q3 of 2015?

- For each year, find what percentage of days it rained (had non-zero precipitation).